In [1]:
import pytezos as ptz
from pytezos.contract.interface import ContractInterface
from pytezos.michelson.types import MichelsonType
from pytezos.crypto.encoding import base58_decode
from pytezos.crypto.key import blake2b_32
from pytezos import Key

In [2]:
import requests
import json

We're using https://packages.ligolang.org/contract/Permit-Cameligo which itself extends `ligo-extendable-fa2` to add a Permit implementation.

In [3]:
!ligo compile contract permit-cameligo/src/main.mligo > permit-contract.tz

In [6]:
!ligo compile contract staking-contract.mligo > staking-contract.tz

# Minting and staking a NFT

We're using a local network for this demo (typically using Flextesa), but this has been tested on Ghostnet as well. On Ghostnet, operations and balance changes seem to be correctly picked by the indexers, even for a non-revealed account.

In [7]:
TEZOS_RPC = "http://localhost:20000"

Let's initialize two clients, one brand new (unrevealed) and one having a positive balance. The latter will play the API's role.

In [8]:
bob_key = Key.generate()

In [9]:
bob = ptz.pytezos.using(TEZOS_RPC, bob_key)

In [10]:
bob.balance()

Decimal('0.000000')

In [14]:
admin_key = Key.from_encoded_key("edsk3QoqBuvdamxouPhin7swCvkQNgq4jP5KZPbwWNnwdZpSpJiEbq")
admin_key.public_key_hash()

'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb'

In [15]:
admin = ptz.pytezos.using(TEZOS_RPC, admin_key)
admin.balance ()

Decimal('1800000.000000')

## Deploying a new FA2 test contract

We're deploying a NFT contract implementing FA2+Permit. Tokens can only be minted by the admin. Once they are minted, they behave like regular NFTs and can be transferred or sold by their owners.

Moreover, users that don't have any tez or don't want to pay directly can sign an off-chain permit for a specific transfer. Other accounts can then store this permit in the contract and execute said transfer before the permit expires.

In [16]:
fa2_contract = ContractInterface.from_file("permit-contract.tz")

## Setting the correct initial storage

We need to configure a few things in the storage for the contract to be usable. We do this entirely from PyTezos instead of Ligo: first because PyTezos infers several values that we don't care about here, but also to make this notebook as self-contained as possible.

In [17]:
fa2_initial_storage = fa2_contract.storage.dummy()
fa2_initial_storage

{'extension': {'admin': 'KT18amZmM5W7qDWVt2pH6uj7sCEd3kbzLrHT',
  'counter': 0,
  'default_expiry': 0,
  'extension': {},
  'max_expiry': 0,
  'permit_expiries': {},
  'permits': {},
  'user_expiries': {}},
 'ledger': {},
 'metadata': {},
 'operators': {},
 'token_metadata': {}}

This NFT contract requires to configure the tokens before origination. This is done by defining the `token_metadata` big map.

In [18]:
def tezos_hex(s):
    return f"0x{bytes(s, 'utf-8').hex()}"

fa2_initial_storage["metadata"] = {
    "": tezos_hex("tezos-storage:m"),
    "m": json.dumps({
        "name": "Dummy token",
        "interfaces": ["TZIP-12"]
    }).encode("utf-8")
}

fa2_initial_storage["token_metadata"] = {
    0: {"token_id": 0, 
        "token_info": {
            "name": tezos_hex("Dummy token 1"),
            "symbol": tezos_hex("DUMY"),
            "decimals": tezos_hex("0"),
            }
    }
}

This contract also requires to set the maximum supply as another big map (see `src/token_total_supply.mligo`):

In [20]:
fa2_initial_storage["extension"]["extension"] = {
    0: 0  # 0 token initially
}

We set the admin's address so we can mint NFTs:

In [21]:
fa2_initial_storage["extension"]["admin"] = admin_key.public_key_hash()

And finally we set the expiry delay for the permits: 
TODO: check that it makes sense

In [22]:
fa2_initial_storage["extension"]["max_expiry"] = 3600
fa2_initial_storage["extension"]["default_expiry"] = 3600

In [23]:
orig = admin.origination(fa2_contract.script(initial_storage=fa2_initial_storage)).autofill().sign().inject(min_confirmations=1)

In [24]:
nft_contract_address = orig["contents"][0]["metadata"]["operation_result"]["originated_contracts"][0]

This is the contract from the admin point of view (PyTezos will call the contract with admin as the signer).

In [25]:
nft_contract = admin.contract(nft_contract_address)
nft_contract.key.public_key_hash()

'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb'

The same address is in the contract's storage:

In [26]:
nft_contract.storage()["extension"]["admin"]

'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb'

## Minting a token for Bob

This part is pretty standard: the admin mints a token for Bob, by using his address. Only the admin can do that, and this is typically something that we would put behind an API.

In [27]:
_ = nft_contract.mint_token([{
    "owner": bob.key.public_key_hash(),
    "token_id": 0,
    "amount_": 100
}]).send(min_confirmations=1)

Bob now has 100 tokens:

In [28]:
nft_contract.storage["ledger"][(bob.key.public_key_hash(), 0)]()

100

## Staking the NFT using a permit

First let's deploy the staking contract, which does nothing except wait for a transfer of FA2s.

In [29]:
staking_contract = ContractInterface.from_file("staking-contract.tz")
staking_storage = staking_contract.storage.dummy()
staking_storage["nft_address"] = nft_contract.address

In [30]:
orig = admin.origination(
    staking_contract.script(initial_storage=staking_storage)
).autofill().sign().inject(min_confirmations=1)
staking_address = orig["contents"][0]["metadata"]["operation_result"]["originated_contracts"][0]
staking_address

'KT1DsegtTF7FyJesKvZ9utJ5FAsfM1f43xQu'

In [31]:
staking_contract = admin.contract(staking_address)

This is where things start getting tricky:
1. While Bob cannot post transactions directly, we can still create a Python object representing a transfer of 10 tokens to the staking contract.
2. PyTezos can compute the Micheline type and expression for us, which we will then serialize using PACK and hash.
3. Using this hash, we build the permit according to TZIP-17. A permit is made of two pairs. The former is the chain ID and the FA2 contract address, to prevent replay attacks. The latter is the permit counter and the hash of the transfer, which of course depends on the sender and the receiver.

This is a bit involved; I've tried to delegate as much work to PyTezos as I could, but I still needed to write some Micheline expressions by hand. This is something we'd like to hide away in a library.

In [32]:
bob_nft_contract = bob.contract(nft_contract.address)

This is what a call to transfer would look like. If we send it, it fails, as bob has no tez.

In [33]:
bob_transfer = bob_nft_contract.transfer([{
    "from_": bob.key.public_key_hash(),
    "txs": [{
        "to_": staking_address,
        "token_id": 0,
        "amount": 10
    }]
}])

Note that a call FA2 transfer expects a _list_ of transfers, and we're only interested in signing the permit for _one_ transfer. In the Micheline format, it would have the following form:

In [34]:
bob_transfer.parameters["value"][0]["args"]

[{'string': 'tz1RMQ4Afx4ZsjKJT7H9y13WgXwAKUhdUdku'},
 [{'prim': 'Pair',
   'args': [{'string': 'KT1DsegtTF7FyJesKvZ9utJ5FAsfM1f43xQu'},
    {'int': '0'},
    {'int': '10'}]}]]

So the type is the following:

In [35]:
nft_contract.entrypoints["transfer"].as_micheline_expr()["args"][0]

{'prim': 'pair',
 'args': [{'prim': 'address', 'annots': ['%from_']},
  {'prim': 'list',
   'annots': ['%txs'],
   'args': [{'prim': 'pair',
     'args': [{'prim': 'address', 'annots': ['%to_']},
      {'prim': 'pair',
       'args': [{'prim': 'nat', 'annots': ['%token_id']},
        {'prim': 'nat', 'annots': ['%amount']}]}]}]}]}

In [36]:
matcher = MichelsonType.match(nft_contract.entrypoints["transfer"].as_micheline_expr()["args"][0])
micheline_encoded = matcher.from_micheline_value(bob_transfer.parameters["value"][0]["args"])
micheline_encoded

(tz1RMQ…dku * [(KT1Dse…xQu * (0 * 10))])

TODO: this is very complicated. Maybe we could add a helper to the Permit contract (off-chain view?) and use it.

In [37]:
transfer_hash = blake2b_32(micheline_encoded.pack()).hexdigest()
transfer_hash

'01daf816bbb41ad32bd70ba3bcbe492ddb2559af95c6530c819aa8b97a09e300'

Now let's compute the hash of a permit, so that Bob can sign it.

In [40]:
permit_hashed_type = {
    'prim': 'pair',
    'args': [
        {
            'prim': 'pair',
            'args': [
                {'prim': 'chain_id'},
                {'prim': 'address'}
            ]
        },
        {
            'prim': 'pair',
            'args': [
                {'prim': 'int'},
                {'prim': 'bytes'}
            ]
        }
        
    ]
}

In [41]:
permit_hashed_args = [
    [
        {"string": nft_contract.shell.block()["chain_id"]},
        {"string": nft_contract.address}
    ],
    [
        {"int": nft_contract.storage()["extension"]["counter"]},
        {"bytes": transfer_hash}
    ]
]

In [42]:
matcher2 = MichelsonType.match(permit_hashed_type)
permit_hash = matcher2.from_micheline_value(permit_hashed_args).pack()
permit_hash.hex()

'05070707070a0000000420560dfd0a00000016017645d51203cbbf6c7976b8704f32c7529f07a60000070700000a0000002001daf816bbb41ad32bd70ba3bcbe492ddb2559af95c6530c819aa8b97a09e300'

Bob also needs to sign this hash.

In [43]:
bob_permit_signature = bob.key.sign(permit_hash)

The admin (or anyone else) can then send the permit to the contract.

In [44]:
_ = nft_contract.permit([(
    bob.key.public_key(),
    bob_permit_signature,
    transfer_hash
)]).send(min_confirmations=1)

Let's check that the permit has correctly been stored:

In [46]:
nft_contract.storage()  # The counter has been incremented

{'extension': {'admin': 'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb',
  'counter': 1,
  'default_expiry': 3600,
  'extension': 4,
  'max_expiry': 3600,
  'permit_expiries': 5,
  'permits': 6,
  'user_expiries': 7},
 'ledger': 8,
 'metadata': 9,
 'operators': 10,
 'token_metadata': 11}

In [47]:
# and the hash of the transfer has been stored in the `permits` big map
nft_contract.storage["extension"]["permits"][(
    bob.key.public_key_hash(),
    transfer_hash
)]()

1692610229

Of course a direct transfer fails because bob still has no tez…

In [53]:
try:
    _ = bob_nft_contract.transfer([{
        "from_": bob.key.public_key_hash(),
        "txs": [{
            "to_": admin.key.public_key_hash(),
            "token_id": 1,
            "amount": 1
        }]
    }]).send(min_confirmations=1)
except ptz.rpc.errors.RpcError as exc:
    print("Bob cannot call the contract because he has no tez!")
    print(exc)

Bob cannot call the contract because he has no tez!
({'id': 'proto.017-PtNairob.implicit.empty_implicit_contract',
  'implicit': 'tz1RMQ4Afx4ZsjKJT7H9y13WgXwAKUhdUdku',
  'kind': 'branch'},)


And the admin cannot just steal the NFT:

In [56]:
try:
    _ = nft_contract.transfer([{
        "from_": bob.key.public_key_hash(),
        "txs": [{
            "to_": admin.key.public_key_hash(),
            "token_id": 0,
            "amount": 10
        }]
    }]).send(min_confirmations=1)
except ptz.rpc.errors.MichelsonError as exc:
    print("The admin is not allowed to steal the tokens!")
    print(exc)

The admin is not allowed to steal the tokens!
({'id': 'proto.017-PtNairob.michelson_v1.script_rejected',
  'kind': 'temporary',
  'location': 1544,
  'with': {'string': 'FA2_NOT_OPERATOR'}},)


But admin can stake Bob's NFTs in the correct contract:

In [57]:
op = staking_contract.stake(10, bob.key.public_key_hash()).send(min_confirmations=1)
print(op.contents)


Properties
.key		tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb
.shell		['http://localhost:20000']
.block_id	head

Payload
{'branch': 'BLSum22e4BftQfAiMJUz8vKW7gKu8nB6UcAwUjT4ZkDDyrHwobE',
 'contents': [{'amount': '0',
               'counter': '5',
               'destination': 'KT1DsegtTF7FyJesKvZ9utJ5FAsfM1f43xQu',
               'fee': '901',
               'gas_limit': '5865',
               'kind': 'transaction',
               'parameters': {'entrypoint': 'stake',
                              'value': {'args': [{'int': '10'},
                                                 {'string': 'tz1RMQ4Afx4ZsjKJT7H9y13WgXwAKUhdUdku'}],
                                        'prim': 'Pair'}},
               'source': 'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb',
               'storage_limit': '167'}],
 'protocol': 'PtNairobiyssHuh87hEhfVBGCVrK3WnS8Z2FT4ymB5tAa4r1nQf',
 'signature': 'sigRm4aRWM7vKhkiEUSJgui13RXQBb6SL1Ff9KBta87sXTHacEzSs36PPykvVAMhY1RZXNwEmG9Eue4UW2xz6k81BpPashvG'}

Hash
ooo2R4pgAGL3wyS

In [61]:
nft_contract.storage["ledger"][(bob.key.public_key_hash(), 0)]()

90

In [62]:
nft_contract.storage["ledger"][(staking_address, 0)]()

10

Finally, the admin cannot replay the staking once the permit has been used:

In [64]:
try:
    staking_contract.stake(10, bob.key.public_key_hash()).send(min_confirmations=1)
except ptz.rpc.errors.MichelsonError:
    print("Cannot stake twice without signing a new permit")

Cannot stake twice without signing a new permit


## Selling the NFT on objkt.com

TODO

# Appendix

Here's the code for the staking contract.

In [65]:
!cat staking-contract.mligo

#import "permit-cameligo/src/main.mligo" "FA2"

type storage = {
  nft_address: address;
  staked: (address, nat) big_map;
}

(* We need to provide the address of the NFT's owner so that the transfer can be done by someone
 * else (we don't rely on Tezos.get_sender ()) *)

[@entry]
let stake (qty, sender: nat * address) (storage: storage): operation list * storage =
  let staked = match Big_map.find_opt sender storage.staked with
    | None -> Big_map.add sender qty storage.staked
    | Some n -> Big_map.update sender (Some (n + qty)) storage.staked
  in
  let contract = match (Tezos.get_contract_opt storage.nft_address: FA2.parameter contract option) with
    | None -> failwith "Invalid NFT contract"
    | Some contract -> contract
  in
  let transfer = [{
    from_ = sender;
    txs = [{
      to_ = Tezos.get_self_address ();
      token_id = 0n;
      amount = qty;
    }]
  }]
  in
  let op = Tezos.transaction (Transfer transfer: FA2.parameter) 0tez contract in
  [op], { storage wit